## MealsCount Algorithm  
  
This notebook details the implementation of an algorithm to groups schools (within a given school district) for maximizing federal funds received through the [**C**ommunity **E**ligiblity **P**rogram](https://www.fns.usda.gov/school-meals/community-eligibility-provision). The groupings generated by the algorithm are near-optimal, optimality being constrained by the need to minimize computational complexity.  
  
### Background  
  
Currently, the Federal government, through the [Food and Nutrition Service](https://www.usda.gov/topics/food-and-nutrition) of the US Dept of Agriculture, offers multiple programs to provide free and/or subsidized meals to school-going children. These programs are targeted at students with low income families. The CEP is one such program.  
  
School districts apply to enrol their schools in CEP once every 4 years (or each year, under certain circumstances). **CEP eligibility criteria** are listed in detail [here](https://www.cde.ca.gov/ls/nu/sn/cepfactsheet.asp). The program allows schools within a school district to enrol individually or in groups (a minimum of 2 schools per group, up to max schools in the district). There is no limit on the number of groups per school district. A school can only be part of one group (?). Further, groups may contain schools of different types (charter, non-charter and so on).  
  
### Problem Statement  
  
While schools enrolled in CEP **must** serve meals to **all** students, the percentage of such meals covered by federal funds is computed based on the Identified Student Percentage (ISP) of each school. Specifically, it is given by the below formula:  
> *__% Meals Covered__* = *__ISP__ X __1.6__*  
  
This implies that in order to be fully (100%) funded the school (or school group) must have an ISP of at least 62.5% (since *62.5 X 1.6 = 100*). For schools (or school groups) with less than 62.5% ISP, the percentage of meals covered by federal funds decreases on a sliding scale until it reaches a minimum of 64% (since a minimum ISP of 40% is required for CEP enrolment, and *40 X 1.6 = 64*). Any meals not funded by CEP will have to be paid for by the student, or by the school itself in case of the student's inability to pay for the same. The latter is more common than not and leaves schools burdened with debt from partially subsidized meals. It is therefore in the school's best interest to meet the 62.5% ISP threshold for full coverage, either by itself, or as part of a school group.  
  
Currently, school groups within a school district are generated manually (through school officials interacting with spreadsheet data). This often results in sub-optimal groupings leading to either many schools not qualifying for CEP entirely, or failing to get adequate funding for meals served.  
  
### The MealsCount Solution    
  
The MealsCount approach to address the sub-optimalities mentioned above is to use an algorithm to generate the school groups. The algorithm is designed with the following optimization criteria:    
  
1. Maximize the percentage of meals funded by CEP, on a per school basis
2. Maximize the number of schools (i.e.: number of students) enrolled in CEP, on a per district basis  
  
In concrete terms, (1) attempts to generate school groups that have an aggregated ISP of 62.5% but not too much lower (or for that matter, too much higher) than that. (2) attemtps to increase the percentage of schools in a CEP eligible group (i.e.: the group's aggregated ISP is 40% or more, ideally no more than 62.5%) such that it is at or near 100% for the school district.

### Algorithm Design   
  
Generating sets of unique groups (i.e.: school groups in our case) from within a large set (i.e.: school district) is, at its core, a combinatrics problem. More specifically, it falls in the realm of [combinatorial optimization](https://en.wikipedia.org/wiki/Combinatorial_optimization). A set with *__n__* elements has *__2<sup>n<sup>__* unique combinations. A typical school district has anywhere from 15-30 schools, resulting in anywhere from 32K to 1B unique groups that would have to be searched for the above optimization criteria. At this size the problem is not trivial but nevertheless manageable. However, it is not uncommon to find school districts with anywhere from a 100 to 1000 schools (e.g.: LA Unified has a 1000+ schools). This leads to an unimaginably large search space rendering a brute-force solution infeasible. Any practical solutions to the problem will only be **near-optimal**.   

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import pprint

In [2]:
import backend_utils as bu
import config_parser as cp

In [3]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

CONFIG_FILE = "config.json"

##### Algorithm Inputs  
  
The algorithm takes the following inputs:  
  
* School District Input: this contains information needed to compute per-school ISP  
* Configuration: this contains school meal rates, ISP thresholds among other information

In [4]:
data_in = bu.mcXLSchoolDistInput(os.path.join(DATADIR,DATAFILE))
df = data_in.to_frame()
df.head(n=3)

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
0,1000001,School NC01,37,4,27,0,0,6,29,5,30,non-charter
1,1000002,School NC02,1111,503,2,7,0,215,527,122,556,non-charter
2,1000003,School NC03,2332,897,2,14,0,440,979,169,1037,non-charter


In [5]:
cfg = cp.mcModelConfig(CONFIG_FILE)
cfg.show()



MealsCount Model Configuration
------------------------------
Version: 1.0
Min CEP Threshold (%): 0.3
Max CEP Threshold (%): 0.625
CEP Rates Table:
         nslp_lunch_free_rate  nslp_lunch_paid_rate  sbp_bkfst_free_rate  \
default                  3.23                  0.31                 1.75   
AK                       5.24                  0.50                 2.79   
HI                       3.78                  0.36                 2.03   
PR                       3.78                  0.36                 2.03   

         sbp_bkfst_paid_rate  
default                 0.30  
AK                      0.45  
HI                      0.34  
PR                      0.34  


##### Compute ISP  
  
The ISP for each school in the school district is computed from the CALPADs school district input data as below:  
> ISP = (Foster + Homeless + Migrant + Direct Certification) / Total Enrollment

In [6]:
# remove aggregated records
df = df[df['school_name']!='total']

In [7]:
s = ((df['foster'] + df['homeless'] + df['migrant'] + df['direct_cert'])/df['total_enrolled']) * 100
df = df.assign(isp=s)
df.loc[:,'isp'] = np.around(df['isp'].astype(np.double),2)

In [8]:
df.head(n=3)

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type,isp
0,1000001,School NC01,37,4,27,0,0,6,29,5,30,non-charter,89.19
1,1000002,School NC02,1111,503,2,7,0,215,527,122,556,non-charter,20.16
2,1000003,School NC03,2332,897,2,14,0,440,979,169,1037,non-charter,19.55


Sort schools within the district by their ISP.

In [9]:
df.sort_values('isp',ascending=True,inplace=True)
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.head(n=3)

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type,isp
0,1000009,School NC08,3031,585,1,7,0,368,676,178,782,non-charter,12.41
1,1000023,School NC23,1712,467,3,3,0,209,503,137,567,non-charter,12.56
2,1000018,School NC18,94,15,2,1,0,9,21,19,36,non-charter,12.77
